In [14]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

Importing the training data to train a random forest model for predictions.

In [15]:
train_path=os.path.join('data', 'processed', 'train.csv')
train_data=pd.read_csv(train_path)
train_data.head()

,id,hi_int_prncp_pd,acc_now_delinq,acc_open_past_24mths,addr_state,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,...,out_prncp_inv,pub_rec,pub_rec_bankruptcies,purpose,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
0,3819,0,0,2,CT,21120.0,Individual,3662,86.1,0,...,3828.54,0,0,debt_consolidation,C1,36,0,65921,0.0,1
1,3820,0,0,3,CA,32400.0,Individual,2973,30.2,0,...,0.00,3,2,debt_consolidation,C1,36,0,17840,0.0,0
2,3821,0,0,3,NY,30251.0,Individual,1983,46.2,0,...,0.00,0,0,debt_consolidation,B5,36,0,19829,0.0,1
3,3822,0,0,1,NJ,66976.0,Individual,1638,77.4,0,...,9665.27,0,0,debt_consolidation,C1,60,0,9830,0.0,1
4,3823,0,0,2,MO,125000.0,Individual,577,0.0,0,...,11358.24,0,0,debt_consolidation,A2,60,0,6924,0.0,1


In [16]:
train_data.describe()

,id,hi_int_prncp_pd,acc_now_delinq,acc_open_past_24mths,annual_inc,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,...,num_tl_90g_dpd_24m,num_tl_30dpd,out_prncp_inv,pub_rec,pub_rec_bankruptcies,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
count,5778.000000,5778.000000,5778.000000,5778.000000,5.778000e+03,5778.000000,5778.000000,5778.000000,5778.000000,5778.000000,...,5778.000000,5778.000000,5778.000000,5778.000000,5778.000000,5778.000000,5778.000000,5.778000e+03,5778.000000,5778.000000
mean,6707.500000,0.186051,0.004673,4.621668,8.717332e+04,14183.183281,56.451142,0.313949,12.646591,17.952603,...,0.084978,0.003288,4409.801099,0.192108,0.116476,42.782970,259.792489,1.519744e+05,1.836976,0.658013
std,1668.109259,0.389181,0.073104,3.234501,1.283365e+05,17126.031813,28.580262,0.902605,537.782505,8.805861,...,0.532365,0.060202,7471.966849,0.562020,0.353183,10.807535,1996.679613,1.752594e+05,15.620245,0.474416
min,3819.000000,0.000000,0.000000,0.000000,1.000000e+04,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,0.000000,3.000000e+00,0.000000,0.000000
25%,5263.250000,0.000000,0.000000,2.000000,5.100000e+04,3218.500000,33.300000,0.000000,0.000000,11.490000,...,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,0.000000,3.086000e+04,0.000000,0.000000
50%,6707.500000,0.000000,0.000000,4.000000,7.000000e+04,7486.500000,57.800000,0.000000,0.000000,17.310000,...,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,0.000000,8.069450e+04,0.000000,1.000000
75%,8151.750000,0.000000,0.000000,6.000000,1.000000e+05,19536.750000,81.575000,0.000000,0.000000,24.020000,...,0.000000,0.000000,6659.367500,0.000000,0.000000,60.000000,0.000000,2.246718e+05,0.000000,1.000000
max,9596.000000,1.000000,2.000000,28.000000,6.200000e+06,251321.000000,140.500000,18.000000,37157.000000,99.660000,...,15.000000,2.000000,39055.410000,9.000000,7.000000,60.000000,95806.000000,2.801964e+06,497.400000,1.000000


In [17]:
train_data.isnull().sum()

id                      0
hi_int_prncp_pd         0
acc_now_delinq          0
acc_open_past_24mths    0
addr_state              0
annual_inc              0
application_type        0
avg_cur_bal             0
bc_util                 0
delinq_2yrs             0
delinq_amnt             0
dti                     0
earliest_cr_line        0
emp_length              1
emp_title               0
grade                   0
home_ownership          0
initial_list_status     0
int_rate                0
last_credit_pull_d      0
loan_amnt               0
mort_acc                0
num_sats                0
num_tl_120dpd_2m        0
num_tl_90g_dpd_24m      0
num_tl_30dpd            0
out_prncp_inv           0
pub_rec                 0
pub_rec_bankruptcies    0
purpose                 0
sub_grade               0
term                    0
tot_coll_amt            0
tot_cur_bal             0
total_rec_late_fee      0
verification_status     0
dtype: int64

The only missing data is located in the employment length column. The test data showed similar behavior.

In [27]:
pd.set_option('display.max_columns', 40)

In [28]:
train_data.loc[train_data['emp_length'].isnull()]

,id,hi_int_prncp_pd,acc_now_delinq,acc_open_past_24mths,addr_state,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,earliest_cr_line,emp_length,emp_title,grade,home_ownership,initial_list_status,int_rate,last_credit_pull_d,loan_amnt,mort_acc,num_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_30dpd,out_prncp_inv,pub_rec,pub_rec_bankruptcies,purpose,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
23,3842,0,0,3,OH,105237.0,Individual,3062,85.3,1,0,16.66,1990,NaN,OWNER,C,MORTGAGE,w,14.65,2019,3000,0,15,0,0,0,0.0,0,0,credit_card,C5,36,76,45933,0.0,1


Building a pipeline to use for the random forest model.

In [72]:
# will first try without using addr_state, purpose and emp_title; naive approach
y_train=train_data['hi_int_prncp_pd'].copy()

x_train_rf=train_data.drop(columns=['addr_state', 'emp_title', 'hi_int_prncp_pd', 'id', 'purpose'])
x_train_rf

,acc_now_delinq,acc_open_past_24mths,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,earliest_cr_line,emp_length,grade,home_ownership,initial_list_status,int_rate,last_credit_pull_d,loan_amnt,mort_acc,num_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_30dpd,out_prncp_inv,pub_rec,pub_rec_bankruptcies,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
0,0,2,21120.0,Individual,3662,86.1,0,0,33.69,2010,1.0,C,RENT,w,12.62,2019,7000,0,18,0,0,0,3828.54,0,0,C1,36,0,65921,0.00,1
1,0,3,32400.0,Individual,2973,30.2,0,0,15.78,1975,3.0,C,OWN,w,11.99,2016,16200,2,8,0,0,0,0.00,3,2,C1,36,0,17840,0.00,0
2,0,3,30251.0,Individual,1983,46.2,0,0,20.04,2010,2.0,B,RENT,w,11.49,2018,9600,0,10,0,0,0,0.00,0,0,B5,36,0,19829,0.00,1
3,0,1,66976.0,Individual,1638,77.4,0,0,8.58,2012,1.0,C,OWN,w,12.62,2019,12000,0,6,0,0,0,9665.27,0,0,C1,60,0,9830,0.00,1
4,0,2,125000.0,Individual,577,0.0,0,0,2.34,1982,10.0,A,MORTGAGE,w,6.49,2019,24000,4,13,0,0,0,11358.24,0,0,A2,60,0,6924,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5773,0,10,72000.0,Individual,12881,89.7,0,0,24.50,2008,3.0,D,RENT,w,18.25,2019,20000,0,13,0,0,0,6070.29,0,0,D3,36,0,167450,253.96,1
5774,0,2,125000.0,Individual,104603,90.2,0,0,6.39,2005,10.0,C,OWN,w,16.01,2019,15000,2,4,0,0,0,13250.14,0,0,C5,60,0,523017,0.00,0
5775,0,3,95000.0,Individual,22815,96.5,0,0,28.45,1993,10.0,B,MORTGAGE,w,11.53,2019,32000,2,18,0,0,0,9188.48,0,0,B5,60,0,410676,0.00,1
5776,0,5,42000.0,Individual,2586,96.3,0,0,18.20,1999,10.0,C,RENT,w,13.67,2019,12000,0,8,0,0,0,5791.18,0,0,C3,60,0,20684,0.00,1


In [73]:
x_train_rf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5778 entries, 0 to 5777
Data columns (total 31 columns):
acc_now_delinq          5778 non-null int64
acc_open_past_24mths    5778 non-null int64
annual_inc              5778 non-null float64
application_type        5778 non-null object
avg_cur_bal             5778 non-null int64
bc_util                 5778 non-null float64
delinq_2yrs             5778 non-null int64
delinq_amnt             5778 non-null int64
dti                     5778 non-null float64
earliest_cr_line        5778 non-null int64
emp_length              5777 non-null float64
grade                   5778 non-null object
home_ownership          5778 non-null object
initial_list_status     5778 non-null object
int_rate                5778 non-null float64
last_credit_pull_d      5778 non-null int64
loan_amnt               5778 non-null int64
mort_acc                5778 non-null int64
num_sats                5778 non-null int64
num_tl_120dpd_2m        5778 non-null int64

In [74]:
num_attribs=list(x_train_rf.select_dtypes(include=['int64', 'float64']))
cat_attribs=list(x_train_rf.select_dtypes(include=['object']))

In [75]:
knn_impute=KNNImputer(n_neighbors=5)

full_pipeline=ColumnTransformer([
    ('num', knn_impute, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [76]:
x_train_prepared=full_pipeline.fit_transform(x_train_rf)
pd.DataFrame(x_train_prepared).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75
0,0.0,2.0,21120.0,3662.0,86.1,0.0,0.0,33.69,2010.0,1.0,12.62,2019.0,7000.0,0.0,18.0,0.0,0.0,0.0,3828.54,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,32400.0,2973.0,30.2,0.0,0.0,15.78,1975.0,3.0,11.99,2016.0,16200.0,2.0,8.0,0.0,0.0,0.0,0.00,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,30251.0,1983.0,46.2,0.0,0.0,20.04,2010.0,2.0,11.49,2018.0,9600.0,0.0,10.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,66976.0,1638.0,77.4,0.0,0.0,8.58,2012.0,1.0,12.62,2019.0,12000.0,0.0,6.0,0.0,0.0,0.0,9665.27,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,125000.0,577.0,0.0,0.0,0.0,2.34,1982.0,10.0,6.49,2019.0,24000.0,4.0,13.0,0.0,0.0,0.0,11358.24,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
params={'n_estimators': [50,100,150,200,250,500],
        'max_features': [2,4,8,12,15,20,25,30,40],
        'max_depth': [4,6,8,10,12,14,20]}

rf_loans=RandomForestClassifier(random_state=402)

# full grid search
rf_models=GridSearchCV(rf_loans,
                      params,
                      scoring='accuracy',
                      n_jobs=-1,
                      cv=10)

In [79]:
rf_models.fit(x_train_prepared, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=402),
             n_jobs=-1,
             param_grid={'max_depth': [4, 6, 8, 10, 12, 14, 20],
                         'max_features': [2, 4, 8, 12, 15, 20, 25, 30, 40],
                         'n_estimators': [50, 100, 150, 200, 250, 500]},
             scoring='accuracy')

In [80]:
rf_models.best_estimator_

RandomForestClassifier(max_depth=20, max_features=40, n_estimators=250,
                       random_state=402)

In [81]:
rf_models.best_score_

0.9184857243947635

The best score through cross validation was 91.8%.

In [82]:
rf_loans_tuned=rf_models.best_estimator_
rf_loans_tuned

RandomForestClassifier(max_depth=20, max_features=40, n_estimators=250,
                       random_state=402)

In [83]:
predictions=rf_loans_tuned.predict(x_train_prepared)
predictions

array([0, 0, 0, ..., 1, 1, 1])

In [84]:
accuracy_score(y_train, predictions)

1.0

Now that we have a tuned model (it is not a final tuned model because the grid search selected 25 as the best max_features and that was the max value set, so might look at other values later), we can make predictions on the test data.

In [85]:
test_path=os.path.join('data', 'processed', 'test.csv')

test_data=pd.read_csv(test_path)
test_data.head()

,id,acc_now_delinq,acc_open_past_24mths,addr_state,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,earliest_cr_line,emp_length,emp_title,grade,home_ownership,initial_list_status,int_rate,last_credit_pull_d,loan_amnt,mort_acc,num_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_30dpd,out_prncp_inv,pub_rec,pub_rec_bankruptcies,purpose,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
0,1,0,10,NY,45000.0,Individual,3125,40.6,0,0,27.60,1987,10.0,Security Guard,D,RENT,w,17.57,2018,10000,0,21,0,0,0,0.00,0,0,medical,D4,60,379,65615,0.0,1
1,2,0,1,NY,40000.0,Individual,1534,26.6,0,0,17.54,2010,3.0,Security Guard,B,OWN,w,8.24,2017,14400,0,10,0,0,0,0.00,0,0,credit_card,B1,36,0,13806,0.0,0
2,3,0,1,FL,113000.0,Individual,49748,82.6,0,0,19.67,1999,8.0,Administrator,A,MORTGAGE,w,7.49,2019,30000,4,9,0,0,0,8141.22,0,0,debt_consolidation,A4,36,0,447731,0.0,1
3,4,0,6,CA,105000.0,Individual,42642,81.8,0,0,30.47,1988,10.0,Administrator,B,MORTGAGE,w,8.18,2019,10000,2,12,0,0,0,0.00,0,0,credit_card,B1,36,0,469067,0.0,0
4,5,0,9,CA,80000.0,Individual,4766,52.2,0,0,22.94,1990,10.0,Administrator,A,RENT,w,5.32,2019,6000,0,21,0,0,0,0.00,0,0,home_improvement,A1,36,0,100087,0.0,0


In [86]:
test_data_dropped=test_data.drop(columns=['addr_state', 'emp_title', 'id', 'purpose'])
test_data_dropped.head()

,acc_now_delinq,acc_open_past_24mths,annual_inc,application_type,avg_cur_bal,bc_util,delinq_2yrs,delinq_amnt,dti,earliest_cr_line,emp_length,grade,home_ownership,initial_list_status,int_rate,last_credit_pull_d,loan_amnt,mort_acc,num_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_30dpd,out_prncp_inv,pub_rec,pub_rec_bankruptcies,sub_grade,term,tot_coll_amt,tot_cur_bal,total_rec_late_fee,verification_status
0,0,10,45000.0,Individual,3125,40.6,0,0,27.60,1987,10.0,D,RENT,w,17.57,2018,10000,0,21,0,0,0,0.00,0,0,D4,60,379,65615,0.0,1
1,0,1,40000.0,Individual,1534,26.6,0,0,17.54,2010,3.0,B,OWN,w,8.24,2017,14400,0,10,0,0,0,0.00,0,0,B1,36,0,13806,0.0,0
2,0,1,113000.0,Individual,49748,82.6,0,0,19.67,1999,8.0,A,MORTGAGE,w,7.49,2019,30000,4,9,0,0,0,8141.22,0,0,A4,36,0,447731,0.0,1
3,0,6,105000.0,Individual,42642,81.8,0,0,30.47,1988,10.0,B,MORTGAGE,w,8.18,2019,10000,2,12,0,0,0,0.00,0,0,B1,36,0,469067,0.0,0
4,0,9,80000.0,Individual,4766,52.2,0,0,22.94,1990,10.0,A,RENT,w,5.32,2019,6000,0,21,0,0,0,0.00,0,0,A1,36,0,100087,0.0,0


In [88]:
test_data_prepared=full_pipeline.transform(test_data_dropped)
pd.DataFrame(test_data_prepared).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75
0,0.0,10.0,45000.0,3125.0,40.6,0.0,0.0,27.60,1987.0,10.0,17.57,2018.0,10000.0,0.0,21.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,40000.0,1534.0,26.6,0.0,0.0,17.54,2010.0,3.0,8.24,2017.0,14400.0,0.0,10.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,113000.0,49748.0,82.6,0.0,0.0,19.67,1999.0,8.0,7.49,2019.0,30000.0,4.0,9.0,0.0,0.0,0.0,8141.22,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,6.0,105000.0,42642.0,81.8,0.0,0.0,30.47,1988.0,10.0,8.18,2019.0,10000.0,2.0,12.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,9.0,80000.0,4766.0,52.2,0.0,0.0,22.94,1990.0,10.0,5.32,2019.0,6000.0,0.0,21.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
test_predictions=rf_loans_tuned.predict(test_data_prepared)
test_predictions

array([0, 0, 0, ..., 1, 1, 1])

In [105]:
id_data=pd.DataFrame(test_data['id'].copy())
rf_predictions=pd.concat([id_data, pd.DataFrame(test_predictions)], axis=1).rename(columns={'id':'Id', 0:'Category'})
rf_predictions

,Id,Category
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
3813,3814,0
3814,3815,0
3815,3816,1
3816,3817,1


In [109]:
rf_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 2 columns):
Id          3818 non-null int64
Category    3818 non-null int64
dtypes: int64(2)
memory usage: 59.8 KB


In [110]:
rf_predictions['Category']=rf_predictions['Category'].apply(str)

In [111]:
rf_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 2 columns):
Id          3818 non-null int64
Category    3818 non-null object
dtypes: int64(1), object(1)
memory usage: 59.8+ KB


In [113]:
predictions_path=os.path.join('data', 'predictions', 'rf_prediction.csv')
rf_predictions.to_csv(predictions_path, index=False)